In [1]:
!pip install darts optuna yfinance 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212

In [ ]:
#Optuna Hyperparameter Optimization

import numpy as np
import optuna
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler

from darts import TimeSeries
from darts.metrics import smape, mape
from darts.models import RNNModel
from darts.utils.likelihood_models import GaussianLikelihood
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller


import yfinance as yf
# #Getting date from one year ago
# one_year_ago = datetime.now() - relativedelta(years=1) 
# one_year_ago = one_year_ago.strftime("%Y-%m-01")

# #Getting date today
# today = datetime.now()
# today = today.strftime("%Y-%m-01")

# #Adding one day to the date one year ago 
# one_year_ago_plus_one = datetime.now() - relativedelta(years=1) + timedelta(days=1)
# one_year_ago_plus_one = one_year_ago_plus_one.strftime("%Y-%m-%d")
# four_years_ago = datetime.now() - relativedelta(years=4)
# four_years_ago = four_years_ago.strftime("%Y-%m-01")
# print(four_years_ago)
# print(today)
# #Accessing YFinance
# ticker= yf.Ticker("fico")

# #Getting pandas dataframe of stock data from one year ago
# df = ticker.history(start=four_years_ago, end=today, interval="1mo")
# print(df)
# idx = pd.date_range(start=one_year_ago, end=today, freq = 'M')
# # print(idx)



# #Making index into Datetime index
# df.index = pd.DatetimeIndex(df.index)
# df.index = df.index.tz_localize(None) 

# #Reindexing dataframe to fill in missing dates due to stock market closing on weekends
# # df = df.reindex(idx, method = 'pad')


# print(df)


## load data and preprocess

#Getting date from one year ago
one_year_ago = datetime.now() - relativedelta(years=1) 
one_year_ago = one_year_ago.strftime("%Y-%m-%d")

#Getting date today
today = datetime.now()
today = today.strftime("%Y-%m-%d")

#Adding one day to the date one year ago 
one_year_ago_plus_one = datetime.now() - relativedelta(years=1) + timedelta(days=1)
one_year_ago_plus_one = one_year_ago_plus_one.strftime("%Y-%m-%d")

#Accessing YFinance
ticker= yf.Ticker("fico")

#Getting pandas dataframe of stock data from one year ago
df = ticker.history(start=one_year_ago, end=today, interval="1d")
idx = pd.date_range(start=one_year_ago_plus_one, end=today)




#Making index into Datetime index
df.index = pd.DatetimeIndex(df.index)
df.index = df.index.tz_localize(None) 

#Reindexing dataframe to fill in missing dates due to stock market closing on weekends
df = df.reindex(idx, method = 'pad')

#Put dataframe into a Darts "Timeseries" object so that data can be fed into a Darts forecasting model. For more information on Timeseries objects, please look here: https://unit8co.github.io/darts/generated_api/darts.timeseries.html 
series = TimeSeries.from_dataframe(df)

#Drop all columns besides "Close" Column
adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])

# split in train / validation (note: in practice we would also need a test set)
VAL_LEN = 36
TEST_LEN = 37
train, val, test = adj_series[0:292], adj_series[292:-TEST_LEN], adj_series[-TEST_LEN:]

# scale
scaler = Scaler(MaxAbsScaler())
train = scaler.fit_transform(train)
val = scaler.transform(val)

# define objective function
def objective(trial):
    # select input and output chunk lengths
    input_chunk_length = trial.suggest_int("input_chunk_length", 7, 28)
    
  
    # Other hyperparameters
    batch_size = trial.suggest_int("batch_size", 16, 64)
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 2, 5)
    hidden_dim = trial.suggest_int("hidden_dim", 1, 10)
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    n_epochs = trial.suggest_float("n_epochs", 10, 100 )
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=3, verbose=True)
    callbacks = [pruner, early_stopper]

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "gpus": -1,
            "auto_select_gpus": True,
            "callbacks": callbacks,
        }
        num_workers = 4
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # optionally also add the (scaled) year value as a past covariate
    # if include_year:
    #     encoders = {"datetime_attribute": {"past": ["year"]},
    #                 "transformer": Scaler()}
    # else:
    #     encoders = None

    # reproducibility
    torch.manual_seed(42)

    # build the LSTM model, set log_tensorboard = False, random_state was = 42
    my_model = RNNModel(
    model="LSTM",
    n_rnn_layers = n_rnn_layers, # Number of LSTM layers
    hidden_dim=hidden_dim,
    dropout=dropout,
    batch_size=batch_size,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-3}, #learning rate
    model_name="Stock_Forecast",
    training_length=30,
    input_chunk_length=input_chunk_length,
    force_reset=True,
    save_checkpoints=True,
    )


    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    # model_val_set = scaler.transform(series[-(VAL_LEN + input_chunk_length) :])

    # train the model
    my_model.fit(
        series=train,
        val_series=val,
        num_loader_workers=num_workers,
    )

    # reload best model over course of training
    my_model = RNNModel.load_from_checkpoint("Stock_Forecast")

    # Evaluate how good it is on the validation set, using sMAPE
    preds = my_model.predict(series=train, n=len(test))
    smapes = smape(test, preds, n_jobs=-1, verbose=True)
    smape_val = np.mean(smapes)

    return smape_val if smape_val != np.nan else float("inf")


# for convenience, print some optimization trials information
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")


# optimize hyperparameters by minimizing the sMAPE on the validation set
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, callbacks=[print_callback])

[I 2023-05-10 03:36:33,266] A new study created in memory with name: no-name-10bb8a6e-e311-4d03-a0e9-4d0a2d6eb659
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]